In [ ]:
'''
------------------------------------------------
Análisis de modelos predictivos en bolsa
Copyright (C) 2024-2025 MegaStorm Systems

This software is provided 'as-is', without any express or implied
warranty.  In no event will the authors be held liable for any damages
arising from the use of this software.

Permission is granted to anyone to use this software for any purpose,
including commercial applications, and to alter it and redistribute it
freely, subject to the following restrictions:

1. The origin of this software must not be misrepresented; you must not
claim that you wrote the original software. If you use this software
in a product, an acknowledgment in the product documentation would be
appreciated but is not required.
2. Altered source versions must be plainly marked as such, and must not be
misrepresented as being the original software.
3. This notice may not be removed or altered from any source distribution.

------------------------------------------------
Preprocesamiento de datos v1.5
------------------------------------------------'''

# Importar librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.dates as mdates
from matplotlib.colors import ListedColormap 
from matplotlib.ticker import FuncFormatter 
import seaborn as sns
from scipy import stats
import math
import yfinance as yf
import matplotlib.colors as mcolors

In [ ]:
# 1. CARGA Y PREPARACIÓN DE DATOS
#nombre_archivo = "Preprocesamiento de datos.csv" # Original, con errores
start_date = "2015-01-05"
end_date = "2025-05-23"
nombre_archivo = "NVDA_2015-01-05_2025-05-23_SA.csv"
datos = pd.read_csv(nombre_archivo)
df = datos.copy()
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

In [ ]:
# ---------------------------------------------------------------------------------------------------------------
# 1. Graficas de "4.8.1	Visualización general de los datos directos"
# ---------------------------------------------------------------------------------------------------------------

# Formato para esta grafica
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_context("talk")

# Crear figura y ejes (enfoque orientado a objetos)
fig, ax = plt.subplots(figsize=(15, 8))

# Graficar la línea del Precio de Cierre en los ejes 'ax'
ax.plot(
    df.index,
    df['Close'],
    color='navy',
    linewidth=1.5 # Un poco más fino puede ser mejor para rangos largos
)
ax.set_xlim(df.index.min(), df.index.max())

# Ticks Mayores: Uno por cada año
major_locator = mdates.YearLocator()  
major_formatter = mdates.DateFormatter('%Y')  

ax.xaxis.set_major_locator(major_locator)
ax.xaxis.set_major_formatter(major_formatter)

# Ticks Menores: Uno por cada trimestre
minor_locator = mdates.MonthLocator(bymonth=[1, 4, 7, 10]) # Meses de inicio de trimestre
ax.xaxis.set_minor_locator(minor_locator)

# Título y etiquetas
ax.set_title(f'Evolución histórica del precio de cierre de NVIDIA (hasta {df.index.max().strftime('%Y-%m-%d')})', fontsize=16)
ax.set_ylabel('Precio de cierre ($)', fontsize=14) 

# Ajustar tamaño de fuente de los ticks de los ejes
ax.tick_params(axis='y', labelsize=12)
ax.tick_params(axis='x', which='major', labelsize=12)

# Rotar las etiquetas de los años para mejor legibilidad si son muchos
plt.setp(ax.get_xticklabels(which='major'), rotation=90, ha='right')

# Configuración de la rejilla (grid)
#ax.grid(True, which='major', axis='x', linestyle='-', linewidth=0.7 ) # Rejilla vertical en años
#ax.grid(True, which='minor', axis='x', linestyle=':', linewidth=0.5, alpha=0.7) # Rejilla vertical en trimestres (sutil)
#ax.grid(True, which='major', axis='y', linestyle='--', linewidth=0.6) # Rejilla horizontal  

plt.tight_layout()
plt.show()

# Restablecer al estilo por defecto
plt.style.use('default')
sns.set_context("notebook")   

In [ ]:
# Formato para esta grafica
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_context("talk")

# Mostramos solo ultimos años
split_date = pd.Timestamp('2024-05-01')

data2 = df.loc[split_date:].copy()

# Preparación de Datos para el Color del Volumen
# Calcula el cambio en el precio de cierre respecto al día anterior
data2['Price Change'] = data2['Close'].diff()

# Determina el color: verde si sube, rojo si baja o se mantiene
color_gain = '#2ECC71' # Verde esmeralda
color_loss = '#E74C3C' # Rojo alizarina

data2['Volume Color'] = data2['Price Change'].apply(lambda x: color_gain if x > 0 else color_loss)
# Asigna un color por defecto (ej. rojo) al primer día ya que no tiene cambio previo
if not data2.empty:
    data2.loc[data2.index[0], 'Volume Color'] = color_loss
    
# Crear figura y ejes: 2 filas, 1 columna. La primera fila (precio) será más alta.
# sharex=True asegura que ambos gráficos compartan el mismo eje X (fechas)
# Crear figura y ejes
fig, axes = plt.subplots(
    2, 1,
    figsize=(15, 8),
    sharex=True,
    gridspec_kw={'height_ratios': [3, 1]}
)

# --- Gráfico Superior: Precio ---
ax_price = axes[0]

ax_price.fill_between(
    data2.index, data2['Low'], data2['High'],
    color='skyblue', alpha=0.4, label='Rango diario (bajo-alto)'
)

ax_price.plot(
    data2.index, data2['Close'],
    color='navy', linewidth=1.5, label='Precio de cierre' 
)

ax_price.set_ylabel('Precio de cierre ($)', fontsize=14)
# Asegúrate que el título refleje el rango de fechas en data2
start_period = data2.index.min().strftime('%Y-%m-%d')
end_period = data2.index.max().strftime('%Y-%m-%d')
ax_price.set_title(f'Evolución de precio y volumen de NVIDIA ({start_period} a {end_period})', fontsize=16) 

ax_price.legend(loc='best', fontsize=14)  
ax_price.grid(axis='y', linestyle='--', alpha=0.7)
ax_price.tick_params(axis='x', labelbottom=False)
ax_price.tick_params(axis='y', labelsize=12)

# --- Gráfico Inferior: Volumen ---
ax_volume = axes[1]

ax_volume.bar(
    data2.index, data2['Volume'],
    color=data2['Volume Color'],
    width=0.8,  
    alpha=0.8
)

def millions_formatter(x, pos):
    'Función para formatear números grandes en millones (M)'
    return f'{x / 1_000_000:.0f}M' # Dividir por 1 millón y añadir 'M'

def thousands_formatter(x, pos):
    'Función para formatear números en miles (K)'
    if x >= 1_000_000: # Si es un millón o más, usar formato 'M' para evitar números muy largos como 150000K
         return f'{x / 1_000_000:.1f}M'.replace('.0', '') # Formato con un decimal si es necesario
    elif x >= 1_000:
        return f'{x / 1_000:.0f}K' # Dividir por 1000 y añadir 'K'
    else:
        return f'{x:.0f}' # Números menores a 1000 se muestran normal

# Aplicar el formateador personalizado al eje Y del volumen
# ax_volume.yaxis.set_major_formatter(mticker.FuncFormatter(thousands_formatter))
# --> Como el volumen de NVDA es muy alto (cientos de millones), usar Millones (M) es más adecuado
ax_volume.yaxis.set_major_formatter(mticker.FuncFormatter(millions_formatter))
ax_volume.set_ylabel('Volumen (Millones)', fontsize=14) # Actualizar etiqueta Y para reflejar M

# --- Configuración Eje X (Fechas) ---
# Reutilizar el formato automático que funcionaba bien para rangos más cortos
major_locator = mdates.AutoDateLocator(minticks=5, maxticks=10)
major_formatter = mdates.DateFormatter('%Y-%m-%d') # Formato claro

ax_volume.xaxis.set_major_locator(major_locator)
ax_volume.xaxis.set_major_formatter(major_formatter)

ax_volume.tick_params(axis='x', rotation=0, labelsize=12)  
ax_volume.tick_params(axis='y', labelsize=12)

ax_volume.grid(False) # Sin rejilla en volumen para limpiar
ax_volume.set_xlim(data2.index.min(), data2.index.max())

# hspace controla el espacio vertical. Un valor más pequeño junta los gráficos.
# Prueba con valores entre 0 y 0.2 hasta que te guste.
plt.subplots_adjust(hspace=0.1)

# tight_layout se llama después de subplots_adjust para ajustar el resto
plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Ajustar rect para título/etiquetas

plt.show()

# Restablecer al estilo por defecto
plt.style.use('default')
sns.set_context("notebook")   

In [ ]:
# ---------------------------------------------------------------------------------------------------------------
# 2. Graficas de "4.8.3 Detección de valores faltantes y duplicados"
# ---------------------------------------------------------------------------------------------------------------

# 2.0. Calculo inicial
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100
missing_summary = pd.DataFrame({
    'Cantidad Faltante': missing_values,
    'Porcentaje Faltante': missing_percentage
})

# 2.1. Heatmap de Porcentaje
missing_counts_per_year = df.isnull().groupby(df.index.year).sum() # Agrupamos el DataFrame booleano (True=faltante) por año y sumamos (True=1, False=0)
missing_counts_per_year = missing_counts_per_year.loc[:, missing_counts_per_year.sum() > 0] # Filtrar columnas que NUNCA tienen valores faltantes y de este modo reducir el heatmap
total_obs_per_year = df.groupby(df.index.year).size()
missing_pct_per_year = missing_counts_per_year.div(total_obs_per_year, axis=0) * 100
if not missing_pct_per_year.empty:
     plt.figure(figsize=(15, 8))
     ax_pct_heatmap = sns.heatmap(
         missing_pct_per_year,
         cmap="Reds",
         annot=True,
         fmt=".1f", # Mostrar con un decimal
         linewidths=.5,
         linecolor='lightgrey',
         cbar_kws={'label': 'Porcentaje de valores faltantes (%)','orientation': 'horizontal', 'shrink': 0.25, 'aspect': 25, 'pad': 0.16}
     )
     plt.title('Mapa de calor de valores faltantes por variable y año (porcentaje)', fontsize=16)
     plt.xlabel("")
     plt.ylabel("")
     plt.xticks(rotation=90, ha='right')
     plt.yticks(rotation=0)
     ax_pct_heatmap.invert_yaxis() 
     plt.tight_layout()
     plt.show()

# 2.2. Visualización de Valores Faltantes (Barras)
missing_counts_filtered = missing_summary[missing_summary['Cantidad Faltante'] > 0]['Cantidad Faltante']
if not missing_counts_filtered.empty:
    missing_counts_sorted = missing_counts_filtered.sort_values(ascending=False)
    plt.figure(figsize=(14, 7))    
    ax_barplot = sns.barplot(x=missing_counts_sorted.index,
                             y=missing_counts_sorted.values,
                             hue=missing_counts_sorted.index, # Asignar x a hue
                             palette='Reds_r', # Usar paleta Reds
                             legend=False) # Desactivar leyenda ya que hue=x
    # Añadir los valores numéricos encima de cada barra
    # Iterar a través de los contenedores de barras generados por seaborn
    for container in ax_barplot.containers:
        ax_barplot.bar_label(container, fmt='%d', label_type='edge', padding=3, fontsize=10) # fmt='%d' para entero, padding añade espacio

    plt.title('Número de valores faltantes por variable', fontsize=16)
    plt.xlabel('', fontsize=12)
    plt.xticks(rotation=90, ha='right')
    # Ajustar límite Y un poco más para asegurar que las etiquetas caben bien
    plt.ylim(0, missing_counts_sorted.max() * 1.15)
    plt.tight_layout()
    plt.show()
else:
    print("\nNo se encontraron valores faltantes significativos para graficar en barras.")
# Imprimir el resumen numérico al final
print("\nResumen de valores faltantes por columna (ordenado):")
print(missing_summary[missing_summary['Cantidad Faltante'] > 0].sort_values(by='Cantidad Faltante', ascending=False))
print("-" * 50) # Separador visual

# 2.3.Detección de Registros Duplicados
# Contar el número total de filas duplicadas
num_duplicates = df.duplicated().sum()
print(f"\nNúmero total de registros duplicados encontrados: {num_duplicates}")
if num_duplicates > 0:
    # Opcional: Mostrar las filas duplicadas (puede ser extenso)
    # print("\nEjemplos de filas duplicadas:")
    # print(df[df.duplicated(keep=False)].sort_index().head()) # keep=False marca todas las duplicadas

    # Crear un resumen visual simple (Tabla o similar)
    duplicate_summary = pd.DataFrame({
        'Condición': ['Registros Únicos', 'Registros Duplicados'],
        'Cantidad': [len(df) - num_duplicates, num_duplicates]
    })
    print("\nResumen de Duplicados:")
    print(duplicate_summary)

    # Opcional: Visualización muy simple con barras
    plt.figure(figsize=(6, 4))
    sns.barplot(x='Condición', y='Cantidad', data=duplicate_summary)
    plt.title('Conteo de Registros Únicos vs. Duplicados')
    plt.ylabel('Número de Registros')
    plt.xlabel('')
    plt.show()

    # 5. Tratamiento de Duplicados: Eliminación
    print(f"\nForma original del DataFrame: {df.shape}")
    df_original_shape = df.shape # Guardar forma original para comparación
    df.drop_duplicates(inplace=True) # inplace=True modifica el df directamente
    print(f"Forma del DataFrame tras eliminar duplicados: {df.shape}")
    if df.shape[0] < df_original_shape[0]:
         print(f"Se eliminaron {df_original_shape[0] - df.shape[0]} registros duplicados.")
    else:
         # Esto no debería pasar si num_duplicates > 0, pero por si acaso
         print("No se eliminaron filas duplicadas (verificar lógica si se detectaron duplicados).")

else:
    print("\nNo se encontraron registros duplicados en el dataset.")
    # Si no hay duplicados, no es necesario ejecutar drop_duplicates ni mostrar gráficos

In [ ]:
# ---------------------------------------------------------------------------------------------------------------
# 3. Graficas de "4.8.4	Distribuciones e histogramas"
# ---------------------------------------------------------------------------------------------------------------

# 3.1. Definición de Grupos de Variables
direct_data_cols = ['Close', 'High', 'Low', 'Open', 'Volume']
indicator_cols = ['SMA_200', 'SMA_50', 'SMA_20', 'EMA_20', 'ATR_14', 'BB_upper', 'BB_lower', 'Range', 
                  'MACD', 'MACD_signal', 'Chaikin_Osc', 'RSI_14', 'OC_Change']
big_tech_cols = ['Google', 'Amazon', 'Apple', 'Meta', 'Microsoft', 'Tesla', 'AMD', 'Intel']
index_cols = ['SP500', 'NASDAQ100', 'EuroStoxx50', 'Nikkei225', 'ShanghaiComposite']
economic_cols = ['Treasury_3M', 'Treasury_10Y', 'VIX', 'Brent_Oil', 'Gold',
                 'CPI', 'GDP_Real', 'GDP_per_Capita']
sentiment_cols = ['googletrends_NVDA', 'av_nvidia']

# Agrupar todas las definiciones
variable_groups = {
    "Datos directos": {'cols': direct_data_cols, 'layout': (2, 3)},
    "Indicadores técnicos": {'cols': indicator_cols, 'layout': (4, 3)},
    "Big Tech": {'cols': big_tech_cols, 'layout': (3, 3)},
    "Índices bursátiles": {'cols': index_cols, 'layout': (2, 3)},
    "Indicadores económicos": {'cols': economic_cols, 'layout': (3, 3)},
    "Análisis de sentimiento": {'cols': sentiment_cols, 'layout': (1, 2)}
}

# 3.2. Mapeo de Unidades
unit_map = {
    'Close': '$', 'Open': '$', 'High': '$', 'Low': '$',
    'Volume': 'Millones de compra/venta',
    'SMA_200': '$', 'SMA_50': '$', 'SMA_20': '$', 'EMA_20': '$',
    'BB_upper': '$', 'BB_lower': '$', 'Range': '$', 'ATR_14': '$',
    'RSI_14': '% 0-100',
    'MACD': 'Valor', 'MACD_signal': 'Valor',
    'OC_Change': '%',
    'Chaikin_Osc': 'Valor',
    'Google': '$', 'Amazon': '$', 'Apple': '$', 'Meta': '$',
    'Microsoft': '$', 'Tesla': '$', 'AMD': '$', 'Intel': '$',
    'SP500': 'Pts', 'NASDAQ100': 'Pts', 'EuroStoxx50': 'Pts',
    'Nikkei225': 'Pts', 'ShanghaiComposite': 'Pts',
    'Treasury_3M': 'Miles de millones $', 'Treasury_10Y': 'Miles de millones $',
    'VIX': 'Pts',
    'Brent_Oil': '$/Barril', 'Gold': '$/Decima parte de onza troy de oro',
    'CPI': '%',
    'GDP_Real': 'Miles de millones $',
    'GDP_per_Capita': '$',
    'googletrends_NVDA': 'Rango',
    'av_nvidia': 'Rango',
}

# 3.3. Función Reutilizable para Graficar Distribuciones  
def plot_distributions(df, columns_to_plot, nrows, ncols, figure_title, cmap_name='Reds', unit_map={}):
    # Filtrar columnas que realmente existen en el DataFrame
    actual_cols = [col for col in columns_to_plot if col in df.columns]
    if not actual_cols:
        print(f"Advertencia: Ninguna de las columnas para '{figure_title}' existe en el DataFrame. Omitiendo gráfico.")
        return

    num_plots = len(actual_cols)
    nrows_actual = math.ceil(num_plots / ncols)
    if nrows_actual == 0: return

    fig, axes = plt.subplots(nrows=nrows_actual, ncols=ncols, figsize=(ncols * 5.5, nrows_actual * 4.5))
    axes = axes.flatten() if num_plots > 1 else [axes]
    fig.suptitle(figure_title, fontsize=18, y=0.97, weight='bold')

    cmap = plt.get_cmap(cmap_name)
    hist_color = cmap(0.7)
    kde_color = cmap(0.95) # Rojo más oscuro para línea KDE (ligeramente más oscuro para contraste)
    label_color = cmap(0.95) # Rojo muy oscuro para etiquetas de ejes
    grid_color = 'grey'

    def millions_formatter(x, pos):
        if x >= 1e6:
             return f'{x*1e-6:.0f}'
        elif x >= 1e3:
             return f'{x*1e-3:.0f}k'
        else:
             return f'{x:.0f}'

    for i, col in enumerate(actual_cols):
        ax = axes[i]
        try:
            sns.histplot(df[col].dropna(),
                         kde=True,
                         ax=ax,
                         bins=30,
                         color=hist_color, # Color sólido para barras
                         alpha=0.6, 
                         line_kws={'color': kde_color, 'linewidth': 2.5} # Línea KDE un poco más gruesa
                        )
            ax.set_title(f'{col}', fontsize=12, weight='bold')

            unit = unit_map.get(col, None)
            xlabel_text = f"{f'({unit})' if unit else ''}"
            ax.set_xlabel(xlabel_text, fontsize=10, color=label_color, weight='medium')

            if i % ncols == 0:
                ax.set_ylabel('Densidad', fontsize=10, color=label_color, weight='medium')
                ax.tick_params(axis='y', labelcolor=label_color)
            else:
                ax.set_ylabel('')
                ax.tick_params(axis='y', labelleft=False)

            ax.tick_params(axis='x', labelcolor=label_color)

            if col == 'Volume':
                 ax.xaxis.set_major_formatter(FuncFormatter(millions_formatter))

            ax.grid(axis='y', linestyle='--', linewidth=0.5, alpha=0.7, color=grid_color)

            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
            ax.spines['left'].set_color(grid_color)
            ax.spines['bottom'].set_color(grid_color)

        except Exception as e:
            print(f"Error graficando {col} en '{figure_title}': {e}")
            ax.set_title(f'Error: {col}', fontsize=11)
            ax.text(0.5, 0.5, 'Error en datos', ha='center', va='center', transform=ax.transAxes)

    # Ocultar ejes sobrantes
    for j in range(num_plots, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout(rect=[0, 0.03, 1, 0.97])
    plt.show()

# 3.4. Generar Gráficos para Cada Grupo
for title, group_info in variable_groups.items():
    plot_distributions(df,
                       group_info['cols'],
                       group_info['layout'][0], # nrows
                       group_info['layout'][1], # ncols
                       title, # Título de la figura
                       cmap_name='Reds', # Mantenemos la paleta base
                       unit_map=unit_map)

In [ ]:
# ---------------------------------------------------------------------------------------------------------------
# 4. Graficas de "4.8.5	Análisis de valores atípicos"
# ---------------------------------------------------------------------------------------------------------------

# Mapeo de Unidades (Asegúrate que sea correcto)
unit_map = {
    'Close': '$', 'Open': '$', 'High': '$', 'Low': '$',
    'Volume': 'Operaciones (Millones)',
    'SMA_200': '$', 'SMA_50': '$', 'SMA_20': '$', 'EMA_20': '$',
    'BB_upper': '$', 'BB_lower': '$', 'Range': '$', 'ATR_14': '$',
    'RSI_14': '% (0-100)',
    'MACD': 'Valor', 'MACD_signal': 'Valor',
    'OC_Change': '%',
    'Chaikin_Osc': 'Valor',
    'Google': '$', 'Amazon': '$', 'Apple': '$', 'Meta': '$',
    'Microsoft': '$', 'Tesla': '$', 'AMD': '$', 'Intel': '$',
    'SP500': 'Pts', 'NASDAQ100': 'Pts', 'EuroStoxx50': 'Pts',
    'Nikkei225': 'Pts', 'ShanghaiComposite': 'Pts',
    'Treasury_3M': '%', 'Treasury_10Y': '%', # Ajustado a %
    'VIX': 'Pts',
    'Brent_Oil': '$/Barril', 'Gold': '$/Decima parte de onza troy de oro',
    'CPI': '%',
    'GDP_Real': 'Miles millones $',
    'GDP_per_Capita': '$',
    'googletrends_NVDA': 'Rango',
    'av_nvidia': 'Rango',
}

# --- Colores Consistentes ---
boxplot_color_main = "#E87C7B"  
# Derivar o elegir colores oscuros para contraste
try:
    # Usar un rojo oscuro de la paleta para outliers si es posible
    cmap = plt.get_cmap('Reds')
    outlier_color_emphasis = cmap(0.9) # Rojo oscuro
    label_tick_color = "#7E0610"  
    graph_color = "#909090"
except Exception as e_cmap:
    print(f"Error al obtener paleta 'Reds', usando colores fallback: {e_cmap}")
    outlier_color_emphasis = "darkred"
    label_tick_color = "#7E0610" 
    graph_color = "#909090"

# Propiedades de Outliers
flier_props = dict(marker='o', markersize=5,
                   markerfacecolor=outlier_color_emphasis,
                   markeredgecolor=outlier_color_emphasis,
                   alpha=0.6)

# Formatter para Volumen
def millions_formatter(y, pos):
    if not np.isfinite(y): return ''
    return f'{y*1e-6:.0f}M' if abs(y) >= 1e6 else f'{y:.0f}'

# Formatter para Miles de Millones (GDP_Real)
def billions_formatter(y, pos):
    if not np.isfinite(y): return ''
    # Ajusta formato según escala real
    return f'{y*1e-9:.1f}B' if abs(y) >= 1e9 else f'{y*1e-6:.1f}M'

# --- Función Auxiliar para Aplicar Estilos Comunes ---
def apply_common_styles(ax, xlabel, ylabel, label_tick_color, apply_yaxis_grid=True):
    """Aplica etiquetas, colores y rejilla a un eje."""
    # ax.set_title(title, fontsize=12, weight='bold', color=label_tick_color) # Título de subplot eliminado
    ax.set_xlabel(xlabel, fontsize=10, color=label_tick_color)
    ax.set_ylabel(ylabel, fontsize=10, color=label_tick_color)
    ax.tick_params(axis='x', colors=label_tick_color, rotation=0) 
    ax.tick_params(axis='y', colors=label_tick_color)
    if apply_yaxis_grid:
        ax.yaxis.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)
    ax.xaxis.grid(False)
    # Opcional: quitar spines superiores/derechos
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_color(graph_color)
    ax.spines['bottom'].set_color(graph_color)

# --- GENERACIÓN DE GRÁFICOS ---
print("Generando Box Plots para análisis de outliers por grupos específicos...")

# 1. Datos Directos (Layout 1x2)
print("\n--- Grupo: Datos Directos ---")
direct_ohlc_cols = ['High', 'Close', 'Low', 'Open']
direct_vol_col = 'Volume'
# Filtrar columnas existentes
valid_ohlc = [c for c in direct_ohlc_cols if c in df.columns and pd.api.types.is_numeric_dtype(df[c])]
has_volume = direct_vol_col in df.columns and pd.api.types.is_numeric_dtype(df[direct_vol_col])

if valid_ohlc or has_volume:
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 5)) # Layout 1x2
    fig.suptitle('Datos directos', fontsize=16, weight='bold', y=1.02)

    # Subplot 0: OHLC
    ax = axes[0]
    if valid_ohlc:
        try:
            # Usar solo 'color' para uniformidad, no 'palette'
            sns.boxplot(data=df[valid_ohlc], ax=ax, color=boxplot_color_main, flierprops=flier_props)
            apply_common_styles(ax, '', unit_map.get('Close', '$'), label_tick_color)
        except Exception as e:
            print(f"Error graficando OHLC: {e}")
            ax.set_title('Error OHLC')
    else:
        ax.set_visible(False)

    # Subplot 1: Volume
    ax = axes[1]
    if has_volume:
        try:
            # Graficar verticalmente y usar Y formatter
            sns.boxplot(y=df[direct_vol_col], ax=ax, color=boxplot_color_main, flierprops=flier_props)
            apply_common_styles(ax, direct_vol_col, unit_map.get(direct_vol_col), label_tick_color)
            ax.yaxis.set_major_formatter(FuncFormatter(millions_formatter))
            ax.set_xticks([]) # Quitar ticks X para plot vertical único
        except Exception as e:
             print(f"Error graficando Volume: {e}")
             ax.set_title('Error Volume')
    else:
         ax.set_visible(False)

    plt.tight_layout(rect=[0, 0.03, 1, 0.99]) # Ajustar rect
    plt.show()
else:
    print("No hay datos válidos para graficar en Datos Directos.")


# 2. Indicadores Técnicos (Layout 3x3)
print("\n--- Grupo: Indicadores Técnicos ---")
# Definir el orden deseado para el grid 3x3
indicator_order = [
    ('single', 'ATR_14', '$'),
    ('single', 'Range', '$'),
    ('multi', ['MACD', 'MACD_signal'], 'Valor', ''), 
    ('single', 'RSI_14', '% (0-100)'),
    ('single', 'OC_Change', '%'),
    ('single', 'Chaikin_Osc', 'Valor'),
    # Última fila
    ('multi', ['SMA_200', 'SMA_50', 'SMA_20', 'EMA_20', 'BB_upper', 'BB_lower'], '$', ''),
]

fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(18, 15)) # Layout 3x3
axes = axes.flatten()
fig.suptitle('Indicadores técnicos', fontsize=16, weight='bold', y=1.02)
plot_index = 0

for i, plot_info in enumerate(indicator_order):
    if plot_index >= len(axes): break # Salir si nos pasamos de ejes
    ax = axes[plot_index]
    plot_type = plot_info[0]
    unit = plot_info[2] if len(plot_info) > 2 else unit_map.get(plot_info[1], 'Valor')

    try:
        if plot_type == 'multi':
            cols = plot_info[1]
            title_label = plot_info[3] # Usar como etiqueta X
            valid_cols = [c for c in cols if c in df.columns and pd.api.types.is_numeric_dtype(df[c])]
            if valid_cols:
                sns.boxplot(data=df[valid_cols], ax=ax, color=boxplot_color_main, flierprops=flier_props)
                apply_common_styles(ax, title_label, unit, label_tick_color)
                ax.tick_params(axis='x', rotation=45, ha='right') # Rotar si son muchos
                plot_index += 1
            else: print(f"Skipping empty multi-plot: {title_label}")
        elif plot_type == 'single':
            col = plot_info[1]
            unit_single = plot_info[2]
            if col in df.columns and pd.api.types.is_numeric_dtype(df[col]):
                sns.boxplot(y=df[col], ax=ax, color=boxplot_color_main, flierprops=flier_props)
                apply_common_styles(ax, col, unit_single, label_tick_color)
                ax.set_xticks([])
                plot_index += 1
            else: print(f"Skipping single plot, column not valid: {col}")

    except Exception as e:
        print(f"Error graficando indicador subplot {i}: {e}")
        # ax.set_title(f"Error {plot_info[1]}") # Sin títulos de subplot
        plot_index += 1

# Ocultar ejes sobrantes
for j in range(plot_index, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout(rect=[0, 0.03, 1, 0.99]) # Ajustar rect
plt.show()


# 3. Big Tech (Gráfico único, como antes)
print("\n--- Grupo: Big Tech ---")
big_tech_cols = ['Google', 'Amazon', 'Apple', 'Meta', 'Microsoft', 'Tesla', 'AMD', 'Intel']
valid_big_tech = [c for c in big_tech_cols if c in df.columns and pd.api.types.is_numeric_dtype(df[c])]
if valid_big_tech:
    plt.figure(figsize=(max(8, len(valid_big_tech) * 1.5), 6))
    ax = plt.gca()
    plt.title('Big Tech', fontsize=16, weight='bold', y=1.02) 
    try:
        sns.boxplot(data=df[valid_big_tech], ax=ax, color=boxplot_color_main, flierprops=flier_props)
        apply_common_styles(ax, 'Acciones Big Tech', unit_map.get('Google','$'), label_tick_color)
        ax.tick_params(axis='x', rotation=45) # Quitado ha='right'
        plt.tight_layout()
        plt.show()
    except Exception as e:
        print(f"Error graficando Big Tech: {e}")
        plt.close()
else:
    print("No hay datos válidos para graficar en Big Tech.")


# 4. Índices Bursátiles (Layout 1x2)
print("\n--- Grupo: Índices Bursátiles ---")
indices_g1_cols = ['SP500', 'EuroStoxx50', 'ShanghaiComposite']
indices_g2_cols = ['NASDAQ100', 'Nikkei225']
valid_g1 = [c for c in indices_g1_cols if c in df.columns and pd.api.types.is_numeric_dtype(df[c])]
valid_g2 = [c for c in indices_g2_cols if c in df.columns and pd.api.types.is_numeric_dtype(df[c])]

if valid_g1 or valid_g2:
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 6)) # Layout 1x2
    fig.suptitle('Índices bursátiles', fontsize=16, weight='bold', y=1.02)

    # Subplot 0: Grupo 1
    ax = axes[0]
    if valid_g1:
        try:
            sns.boxplot(data=df[valid_g1], ax=ax, color=boxplot_color_main, flierprops=flier_props)
            apply_common_styles(ax, '', unit_map.get('SP500','Pts'), label_tick_color)
        except Exception as e:
             print(f"Error graficando Índices G1: {e}")
             ax.set_title('Error Índices G1')
    else:
        ax.set_visible(False)

    # Subplot 1: Grupo 2
    ax = axes[1]
    if valid_g2:
        try:
            sns.boxplot(data=df[valid_g2], ax=ax, color=boxplot_color_main, flierprops=flier_props)
            apply_common_styles(ax, '', unit_map.get('NASDAQ100','Pts'), label_tick_color)
        except Exception as e:
             print(f"Error graficando Índices G2: {e}")
             ax.set_title('Error Índices G2')
    else:
        ax.set_visible(False)

    plt.tight_layout(rect=[0, 0.03, 1, 0.99]) # Ajustar rect
    plt.show()
else:
     print("No hay datos válidos para graficar en Índices Bursátiles.")


# 5. Indicadores Económicos (Layout 2x4)
print("\n--- Grupo: Indicadores Económicos ---")
economic_cols = ['Treasury_3M', 'Treasury_10Y', 'VIX', 'Brent_Oil', 'Gold', 'CPI', 'GDP_Real', 'GDP_per_Capita']
valid_economic_cols = [c for c in economic_cols if c in df.columns and pd.api.types.is_numeric_dtype(df[c])]

# --- Función específica para este layout ---
def plot_economic_subplots(df, cols, layout, fig_title, unit_map, box_color, flier_props, label_tick_color):
    if not cols:
        print(f"Advertencia: No hay columnas válidas para graficar en '{fig_title}'.")
        return
    nrows, ncols = layout
    num_plots = len(cols)
    # Ajustar filas necesarias
    nrows = math.ceil(num_plots / ncols)

    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols * 5, nrows * 4.5)) # Ajustar tamaño si es necesario
    axes = axes.flatten() if num_plots > 1 else [axes]
    fig.suptitle(fig_title, fontsize=16, weight='bold', y=1.03)

    for i, col in enumerate(cols):
        ax = axes[i]
        unit = unit_map.get(col, 'Valor')
        try:
            sns.boxplot(y=df[col], ax=ax, color=box_color, flierprops=flier_props)
            apply_common_styles(ax, col, unit, label_tick_color) # Usar col como etiqueta X
            ax.set_xticks([]) # Quitar ticks X

            # Aplicar formato especial si es necesario (GDP?)
            if col in ['GDP_Real']: # Ejemplo
                 try:
                     ax.yaxis.set_major_formatter(FuncFormatter(billions_formatter))
                 except Exception as e_fmt:
                     print(f"Error aplicando formato a eje Y de {col}: {e_fmt}")
            elif col == 'Volume': # Aunque no está en este grupo, por si acaso
                 try:
                     ax.yaxis.set_major_formatter(FuncFormatter(millions_formatter))
                 except Exception as e_fmt:
                      print(f"Error aplicando formato a eje Y de {col}: {e_fmt}")

        except Exception as e:
            print(f"Error generando subplot '{col}' para '{fig_title}': {e}")
            ax.set_title(f'Error: {col}', fontsize=11) # Sin título de subplot

    # Ocultar ejes sobrantes
    for j in range(num_plots, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout(rect=[0, 0.03, 1, 0.99])
    plt.show()
# --- Llamada a la función ---
if valid_economic_cols:
    # Layout 2x4 como solicitado
    plot_economic_subplots(df, valid_economic_cols, (2, 4), 'Indicadores económicos', unit_map, boxplot_color_main, flier_props, label_tick_color)
else:
    print("No se encontraron columnas válidas para graficar Indicadores Económicos.")


# 6. Análisis de sentimiento (Layout 1x2)
print("\n--- Grupo: Análisis de sentimiento ---")
sentiment_cols = ['googletrends_NVDA', 'av_nvidia']

# --- Función específica para este layout ---
def plot_sentiment_subplots(df, cols, layout, fig_title, unit_map, box_color, flier_props, label_tick_color):
    if not cols:
        print(f"Advertencia: No hay columnas válidas para graficar en '{fig_title}'.")
        return
    nrows, ncols = layout
    num_plots = len(cols)
    # Ajustar filas necesarias
    nrows = math.ceil(num_plots / ncols)

    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols * 5, nrows * 4.5)) # Ajustar tamaño si es necesario
    axes = axes.flatten() if num_plots > 1 else [axes]
    fig.suptitle(fig_title, fontsize=16, weight='bold', y=1.03)

    for i, col in enumerate(cols):
        ax = axes[i]
        unit = unit_map.get(col, 'Valor')
        try:
            sns.boxplot(y=df[col], ax=ax, color=box_color, flierprops=flier_props)
            apply_common_styles(ax, col, unit, label_tick_color) # Usar col como etiqueta X
            ax.set_xticks([]) # Quitar ticks X

            # Aplicar formato especial si es necesario (GDP?)
            if col in ['GDP_Real']: # Ejemplo
                 try:
                     ax.yaxis.set_major_formatter(FuncFormatter(billions_formatter))
                 except Exception as e_fmt:
                     print(f"Error aplicando formato a eje Y de {col}: {e_fmt}")
            elif col == 'Volume': # Aunque no está en este grupo, por si acaso
                 try:
                     ax.yaxis.set_major_formatter(FuncFormatter(millions_formatter))
                 except Exception as e_fmt:
                      print(f"Error aplicando formato a eje Y de {col}: {e_fmt}")

        except Exception as e:
            print(f"Error generando subplot '{col}' para '{fig_title}': {e}")
            ax.set_title(f'Error: {col}', fontsize=11) # Sin título de subplot

    # Ocultar ejes sobrantes
    for j in range(num_plots, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout(rect=[0, 0.03, 1, 0.99])
    plt.show()

# --- Llamada a la función ---
# Layout 2x4 como solicitado
plot_sentiment_subplots(df, sentiment_cols, (1, 2), 'Análisis de sentimiento', unit_map, boxplot_color_main, flier_props, label_tick_color)


In [ ]:
# Grafica para META y sus outliers (que no lo son)

# Formato para esta grafica
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_context("talk")

ticker_symbol = "META"
meta_ticker = yf.Ticker(ticker_symbol)
meta_history_full = meta_ticker.history(start=start_date, end=end_date, interval="1d")

# Crear figura y ejes (enfoque orientado a objetos)
fig, ax = plt.subplots(figsize=(15, 8))

# Graficar la línea del Precio de Cierre en los ejes 'ax'
ax.plot(
    meta_history_full.index,
    meta_history_full['Close'],
    color='navy',
    linewidth=1.5 # Un poco más fino puede ser mejor para rangos largos
)

# Ticks Mayores: Uno por cada año
major_locator = mdates.YearLocator()  
major_formatter = mdates.DateFormatter('%Y')  

ax.xaxis.set_major_locator(major_locator)
ax.xaxis.set_major_formatter(major_formatter)

# Ticks Menores: Uno por cada trimestre
minor_locator = mdates.MonthLocator(bymonth=[1, 4, 7, 10]) # Meses de inicio de trimestre
ax.xaxis.set_minor_locator(minor_locator)

# Título y etiquetas
ax.set_title(f'Evolución histórica del precio de cierre de META (hasta {meta_history_full.index.max().strftime('%Y-%m-%d')})', fontsize=16)
ax.set_ylabel('Precio de cierre ($)', fontsize=14) 

# Ajustar tamaño de fuente de los ticks de los ejes
ax.tick_params(axis='y', labelsize=12)
ax.tick_params(axis='x', which='major', labelsize=12)

# Rotar las etiquetas de los años para mejor legibilidad si son muchos
plt.setp(ax.get_xticklabels(which='major'), rotation=0, ha='center')

plt.tight_layout()
plt.show()

# Restablecer al estilo por defecto
plt.style.use('default')
sns.set_context("notebook")   

In [ ]:
# Grafica para EuroStoxx50 y sus outliers (que no lo son)

# Formato para esta grafica
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_context("talk")

ticker_symbol = "^STOXX50E"
meta_ticker = yf.Ticker(ticker_symbol)
meta_history_full = meta_ticker.history(start=start_date, end=end_date, interval="1d")

# Crear figura y ejes (enfoque orientado a objetos)
fig, ax = plt.subplots(figsize=(15, 8))

# Graficar la línea del Precio de Cierre en los ejes 'ax'
ax.plot(
    meta_history_full.index,
    meta_history_full['Close'],
    color='navy',
    linewidth=1.5 # Un poco más fino puede ser mejor para rangos largos
)

# Ticks Mayores: Uno por cada año
major_locator = mdates.YearLocator()  
major_formatter = mdates.DateFormatter('%Y')  

ax.xaxis.set_major_locator(major_locator)
ax.xaxis.set_major_formatter(major_formatter)

# Ticks Menores: Uno por cada trimestre
minor_locator = mdates.MonthLocator(bymonth=[1, 4, 7, 10]) # Meses de inicio de trimestre
ax.xaxis.set_minor_locator(minor_locator)

# Título y etiquetas
ax.set_title(f'Evolución histórica del índice EuroStoxx50 (hasta {meta_history_full.index.max().strftime('%Y-%m-%d')})', fontsize=16)
ax.set_ylabel('Puntos', fontsize=14) 

# Ajustar tamaño de fuente de los ticks de los ejes
ax.tick_params(axis='y', labelsize=12)
ax.tick_params(axis='x', which='major', labelsize=12)

# Rotar las etiquetas de los años para mejor legibilidad si son muchos
plt.setp(ax.get_xticklabels(which='major'), rotation=0, ha='center')

plt.tight_layout()
plt.show()

# Restablecer al estilo por defecto
plt.style.use('default')
sns.set_context("notebook")   

In [ ]:
# Grafica para ShanghaiComposite y sus outliers (que no lo son)

# Formato para esta grafica
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_context("talk")

ticker_symbol = "000001.SS"
meta_ticker = yf.Ticker(ticker_symbol)
meta_history_full = meta_ticker.history(start=start_date, end=end_date, interval="1d")

# Crear figura y ejes (enfoque orientado a objetos)
fig, ax = plt.subplots(figsize=(15, 8))

# Graficar la línea del Precio de Cierre en los ejes 'ax'
ax.plot(
    meta_history_full.index,
    meta_history_full['Close'],
    color='navy',
    linewidth=1.5 # Un poco más fino puede ser mejor para rangos largos
)

# Ticks Mayores: Uno por cada año
major_locator = mdates.YearLocator()  
major_formatter = mdates.DateFormatter('%Y')  

ax.xaxis.set_major_locator(major_locator)
ax.xaxis.set_major_formatter(major_formatter)

# Ticks Menores: Uno por cada trimestre
minor_locator = mdates.MonthLocator(bymonth=[1, 4, 7, 10]) # Meses de inicio de trimestre
ax.xaxis.set_minor_locator(minor_locator)

# Título y etiquetas
ax.set_title(f'Evolución histórica del índice ShanghaiComposite (hasta {meta_history_full.index.max().strftime('%Y-%m-%d')})', fontsize=16)
ax.set_ylabel('Puntos', fontsize=14) 

# Ajustar tamaño de fuente de los ticks de los ejes
ax.tick_params(axis='y', labelsize=12)
ax.tick_params(axis='x', which='major', labelsize=12)

# Rotar las etiquetas de los años para mejor legibilidad si son muchos
plt.setp(ax.get_xticklabels(which='major'), rotation=0, ha='center')

plt.tight_layout()
plt.show()

# Restablecer al estilo por defecto
plt.style.use('default')
sns.set_context("notebook") 

In [ ]:
# Grafica para Nikkei225 y sus outliers (que no lo son)

# Formato para esta grafica
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_context("talk")

ticker_symbol = "^N225"
meta_ticker = yf.Ticker(ticker_symbol)
meta_history_full = meta_ticker.history(start=start_date, end=end_date, interval="1d")

# Crear figura y ejes (enfoque orientado a objetos)
fig, ax = plt.subplots(figsize=(15, 8))

# Graficar la línea del Precio de Cierre en los ejes 'ax'
ax.plot(
    meta_history_full.index,
    meta_history_full['Close'],
    color='navy',
    linewidth=1.5 # Un poco más fino puede ser mejor para rangos largos
)

# Ticks Mayores: Uno por cada año
major_locator = mdates.YearLocator()  
major_formatter = mdates.DateFormatter('%Y')  

ax.xaxis.set_major_locator(major_locator)
ax.xaxis.set_major_formatter(major_formatter)

# Ticks Menores: Uno por cada trimestre
minor_locator = mdates.MonthLocator(bymonth=[1, 4, 7, 10]) # Meses de inicio de trimestre
ax.xaxis.set_minor_locator(minor_locator)

# Título y etiquetas
ax.set_title(f'Evolución histórica del índice Nikkei225 (hasta {meta_history_full.index.max().strftime('%Y-%m-%d')})', fontsize=16)
ax.set_ylabel('Puntos', fontsize=14) 

# Ajustar tamaño de fuente de los ticks de los ejes
ax.tick_params(axis='y', labelsize=12)
ax.tick_params(axis='x', which='major', labelsize=12)

# Rotar las etiquetas de los años para mejor legibilidad si son muchos
plt.setp(ax.get_xticklabels(which='major'), rotation=0, ha='center')

plt.tight_layout()
plt.show()

# Restablecer al estilo por defecto
plt.style.use('default')
sns.set_context("notebook") 

In [ ]:
# Grafica para Gold y sus outliers (que no lo son)

# Formato para esta grafica
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_context("talk")

ticker_symbol = "GLD"
meta_ticker = yf.Ticker(ticker_symbol)
meta_history_full = meta_ticker.history(start=start_date, end=end_date, interval="1d")

# Crear figura y ejes (enfoque orientado a objetos)
fig, ax = plt.subplots(figsize=(15, 8))

# Graficar la línea del Precio de Cierre en los ejes 'ax'
ax.plot(
    meta_history_full.index,
    meta_history_full['Close'],
    color='navy',
    linewidth=1.5 # Un poco más fino puede ser mejor para rangos largos
)

# Ticks Mayores: Uno por cada año
major_locator = mdates.YearLocator()  
major_formatter = mdates.DateFormatter('%Y')  

ax.xaxis.set_major_locator(major_locator)
ax.xaxis.set_major_formatter(major_formatter)

# Ticks Menores: Uno por cada trimestre
minor_locator = mdates.MonthLocator(bymonth=[1, 4, 7, 10]) # Meses de inicio de trimestre
ax.xaxis.set_minor_locator(minor_locator)

# Título y etiquetas
ax.set_title(f'Evolución histórica del precio del oro (hasta {meta_history_full.index.max().strftime('%Y-%m-%d')})', fontsize=16)
ax.set_ylabel('$/Decima parte de onza troy de oro', fontsize=14) 

# Ajustar tamaño de fuente de los ticks de los ejes
ax.tick_params(axis='y', labelsize=12)
ax.tick_params(axis='x', which='major', labelsize=12)

# Rotar las etiquetas de los años para mejor legibilidad si son muchos
plt.setp(ax.get_xticklabels(which='major'), rotation=0, ha='center')

plt.tight_layout()
plt.show()

# Restablecer al estilo por defecto
plt.style.use('default')
sns.set_context("notebook") 

In [ ]:
# ---------------------------------------------------------------------------------------------------------------
# 4. Graficas de "4.8.6	Correlaciones entre variables"
# ---------------------------------------------------------------------------------------------------------------

target_variable = 'Close' 
label_tick_color = "#7E0610" 

# Definición de Grupos de Variables
direct_data_cols = ['Close', 'High', 'Low', 'Open', 'Volume']
indicator_cols = ['SMA_200', 'SMA_50', 'SMA_20', 'EMA_20', 'ATR_14', 'BB_upper', 'BB_lower', 'Range', 
                  'MACD', 'MACD_signal', 'Chaikin_Osc', 'RSI_14', 'OC_Change']
big_tech_cols = ['Google', 'Amazon', 'Apple', 'Meta', 'Microsoft', 'Tesla', 'AMD', 'Intel']
index_cols = ['SP500', 'NASDAQ100', 'EuroStoxx50', 'Nikkei225', 'ShanghaiComposite']
economic_cols = ['Treasury_3M', 'Treasury_10Y', 'VIX', 'Brent_Oil', 'Gold',
                 'CPI', 'GDP_Real', 'GDP_per_Capita']
sentiment_cols = ['googletrends_NVDA', 'av_nvidia']

# Agrupar todas las definiciones
variable_groups = {
    "Datos directos": {'cols': direct_data_cols, 'layout': (2, 3)},
    "Indicadores técnicos": {'cols': indicator_cols, 'layout': (4, 3)},
    "Big Tech": {'cols': big_tech_cols, 'layout': (3, 3)},
    "Índices bursátiles": {'cols': index_cols, 'layout': (2, 3)},
    "Indicadores económicos": {'cols': economic_cols, 'layout': (3, 3)},
    "Análisis de sentimiento": {'cols': sentiment_cols, 'layout': (1, 2)}
}

# --- 1. Heatmaps por Grupo (Incluyendo 'Close') ---
print("Generando Heatmaps de Correlación por Grupo (incluyendo Target)...")

for group_title, group_info in variable_groups.items():
    print(f"\n--- Grupo: {group_title} ---")
    cols_in_group = group_info['cols']

    # Seleccionar columnas válidas del grupo + la variable objetivo
    relevant_cols = [col for col in cols_in_group if col in df.columns and pd.api.types.is_numeric_dtype(df[col])]
    if target_variable not in relevant_cols and target_variable in df.columns and pd.api.types.is_numeric_dtype(df[target_variable]):
        relevant_cols = [target_variable] + relevant_cols # Poner Close al principio
    elif target_variable not in df.columns or not pd.api.types.is_numeric_dtype(df[target_variable]):
        print(f"Advertencia: Variable objetivo '{target_variable}' no encontrada o no numérica. Correlación con target no se mostrará.")
        # relevant_cols sigue siendo solo las del grupo

    if len(relevant_cols) < 2:
        print("No hay suficientes columnas numéricas válidas en este grupo para calcular correlación.")
        continue

    # Calcular correlación solo para este subconjunto
    sub_corr = df[relevant_cols].corr()

    # Crear máscara para la mitad superior
    mask = np.triu(np.ones_like(sub_corr, dtype=bool))

    # Determinar tamaño de figura dinámicamente
    n_vars = len(relevant_cols)
    fig_size = max(8, n_vars * 0.8) # Ajustar multiplicador si es necesario

    plt.figure(figsize=(fig_size, fig_size * 0.8)) # Hacerla un poco más ancha que alta

    sns.heatmap(sub_corr,
                mask=mask,
                cmap='coolwarm',
                vmax=1.0, vmin=-1.0, center=0,
                annot=True, # Puede ser manejable en grupos más pequeños
                fmt=".2f",
                linewidths=.5,
                linecolor='lightgray',
                cbar_kws={"shrink": .7})

    plt.title(f'Correlación - {group_title} (y {target_variable})', fontsize=14, weight='bold')
    plt.xticks(rotation=45, ha='right', fontsize=9)
    plt.yticks(rotation=0, fontsize=9)
    plt.tight_layout()
    plt.show()

In [ ]:
# --- 2. Correlación de Todas las Variables con el Target (Gráfico de Barras) ---
cols_to_drop_for_corr = ['Trend', 'Day', 'Year', 'Month', 'Month_sin', 'Month_cos'] 

if target_variable in df.columns and pd.api.types.is_numeric_dtype(df[target_variable]):
    # 1. Seleccionar solo columnas numéricas y hacer una copia
    df_numeric_filtered = df.select_dtypes(include=np.number).copy()

    # 2. Descartar las columnas especificadas si existen en el DataFrame numérico
    existing_cols_to_drop = [col for col in cols_to_drop_for_corr if col in df_numeric_filtered.columns]
    if existing_cols_to_drop:
        df_numeric_filtered.drop(columns=existing_cols_to_drop, inplace=True)
        print(f"Descartadas las columnas: {existing_cols_to_drop} para el gráfico de correlación con target.")

    # Verificar si la variable objetivo sigue presente después del drop (no debería ser dropeada, pero por si acaso)
    if target_variable not in df_numeric_filtered.columns:
        print(f"Advertencia: La variable objetivo '{target_variable}' fue descartada o no es numérica. No se puede generar gráfico.")
    elif df_numeric_filtered.shape[1] < 2: # Necesitamos al menos el target y otra variable
        print("No hay suficientes columnas numéricas (después de descartar) para calcular correlación con el target.")
    else:
        # Calcular correlación de las variables restantes con el target
        all_corr_with_target = df_numeric_filtered.corr()[target_variable].sort_values(ascending=False)

        # Excluir la correlación consigo misma (que es 1)
        all_corr_with_target = all_corr_with_target.drop(target_variable, errors='ignore') # errors='ignore' por si ya fue dropeada

        if not all_corr_with_target.empty:
            plt.figure(figsize=(10, max(6, len(all_corr_with_target) * 0.35))) # Ajustar altura un poco más

            # Crear gráfico de barras horizontal
            bars = plt.barh(all_corr_with_target.index, all_corr_with_target.values,
                            color=['#E87C7B' if x > 0 else '#87CEEB' for x in all_corr_with_target.values])

            # Añadir etiquetas de valor
            plt.bar_label(bars, fmt='%.2f', padding=3, fontsize=9, color=label_tick_color)

            plt.xlabel(f'Correlación de Pearson con {target_variable}', fontsize=10, color=label_tick_color)
            plt.ylabel('Variable Predictora', fontsize=10, color=label_tick_color)
            plt.title(f'Correlación de todas las variables con {target_variable}', fontsize=14, weight='bold')
            plt.xticks(color=label_tick_color)
            plt.yticks(fontsize=9, color=label_tick_color) # Ajustar tamaño de fuente si es necesario
            plt.axvline(0, color='grey', linewidth=0.8, linestyle='--') # Línea en cero
            plt.grid(axis='x', linestyle='--', alpha=0.6)
            plt.gca().spines['top'].set_visible(False)
            plt.gca().spines['right'].set_visible(False)
            plt.gca().spines['left'].set_color(label_tick_color)
            plt.gca().spines['bottom'].set_color(label_tick_color)
            plt.tight_layout()
            plt.show()
        else:
            print(f"No se encontraron otras variables (después de filtrar y excluir target) para calcular correlación con '{target_variable}'.")
else:
    print(f"Variable objetivo '{target_variable}' no encontrada o no numérica. No se puede generar gráfico de correlación con target.")

In [ ]:
# --- 3. (Opcional) Heatmap Completo sin Anotaciones ---
print("\n--- (Opcional) Visión General - Heatmap Completo sin Anotaciones ---")
try:
    # Seleccionar solo columnas numéricas
    df_numeric_for_corr = df.select_dtypes(include=np.number).copy()

    existing_cols_to_drop = [col for col in cols_to_drop_for_corr if col in df_numeric_for_corr.columns]
    if existing_cols_to_drop:
        df_numeric_for_corr.drop(columns=existing_cols_to_drop, inplace=True)
        print(f"Descartadas las columnas: {existing_cols_to_drop} para el heatmap de correlación completo.")
    
    full_corr_matrix = df_numeric_for_corr.corr()
    mask_full = np.triu(np.ones_like(full_corr_matrix, dtype=bool))

    n_vars = full_corr_matrix.shape[0]
    fig_width = max(15, n_vars * 0.35)
    fig_height = max(12, n_vars * 0.30)

    # Crear figura y ejes principales
    fig, ax = plt.subplots(figsize=(fig_width, fig_height))

    # Crear un eje adicional encima del heatmap (para la colorbar)
    # Coordenadas: [x, y, ancho, alto] en proporción (0 a 1) respecto a la figura
    # Este ejemplo lo coloca en la parte superior derecha sobre la diagonal vacía
    cbar_ax = fig.add_axes([0.45, 0.83, 0.5, 0.015])  # [left, bottom, width, height]

    # Dibujar el heatmap sin barra lateral
    sns.heatmap(
        full_corr_matrix,
        mask=mask_full,
        cmap='seismic',
        center=0,
        annot=False,
        linewidths=.2,
        linecolor='lightgray',
        ax=ax,
        cbar_ax=cbar_ax,
        cbar_kws={"orientation": "horizontal", "label": ""}
    )

    # Configurar la colorbar para que esté bien alineada
    cbar_ax.xaxis.set_ticks_position('top')
    cbar_ax.xaxis.set_label_position('top')
    cbar_ax.tick_params(labelsize=8, length=0)

    # Título y etiquetas
    ax.set_title('Visión general de correlación', fontsize=16, weight='bold')
    fontsize_ticks = max(6, 10 - n_vars // 10)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=60, ha='right', fontsize=fontsize_ticks)
    ax.set_yticklabels(ax.get_yticklabels(), rotation=0, fontsize=fontsize_ticks)

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

except Exception as e:
    print(f"Error generando heatmap completo opcional: {e}")

In [ ]:
# ---------------------------------------------------------------------------------------------------------------
# 5. Graficas de "4.8 Campos seleccionados"
# NOTA: al final no usamos esta tabla si una creada en Excel
# ---------------------------------------------------------------------------------------------------------------

cols_to_drop_for_final = ['Trend', 'Day', 'Year', 'Month', 'Month_sin', 'Month_cos', 'index', 'Ticker', 'Close'] 

# Seleccionar columnas que SÍ quieres mostrar
cols = [c for c in df.columns if c not in cols_to_drop_for_final]

# Repartirlas en una tabla de n_cols columnas
n_cols   = 8                                 
n_rows   = int(np.ceil(len(cols) / n_cols))  
pad      = n_rows * n_cols - len(cols)       
cols_pad = cols + [''] * pad                 
matrix   = np.array(cols_pad).reshape(n_rows, n_cols)

table_df = pd.DataFrame(matrix)          
table_df = table_df.rename_axis(None, axis=1)  

fecha_min = df.index.min().strftime('%Y-%m-%d')
fecha_max = df.index.max().strftime('%Y-%m-%d')

# Crear Styler
styled_df = (
    table_df.style
        .set_caption(f"Variables utilizadas para predecir Close - Rango temporal de {fecha_min} a {fecha_max}")  
        .hide(axis="columns")                       
        .set_table_styles([            
            {
                "selector": "caption",
                "props": [
                    ("caption-side", "top"),
                    ("background-color", "#C0504D"),
                    ("color", "white"),
                    ("font-weight", "bold"),
                    ("text-align", "center"),
                    ("padding", "6px")
                ]
            },            
            {
                "selector": "th",
                "props": [
                    ("background-color", "#C0504D"),
                    ("color", "white"),
                    ("text-align", "center")
                ]
            },            
            {
                "selector": "tbody tr:nth-child(even)",
                "props": [("background-color", "white"), ("color", "black")]
            },            
            {
                "selector": "tbody tr:nth-child(odd)",
                "props": [("background-color", "#F2DCDB"), ("color", "black")]
            }
        ])
        .set_properties(**{
            "text-align": "center",
            "border": "1px solid #DA9694"
        })
        .hide(axis="index")                       
)

styled_df   